In [1]:
# ! pip install pyspark
# ! pip install kafka-python-ng

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *



import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
import time
import random
import numpy as np

In [17]:
spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
# KAFKA_TOPIC_NAME_CONS = "Topic encryptdecrypt"
KAFKA_TOPIC_NAME_CONS = "Topic1"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'



In [19]:
if __name__ == "__main__":
    print("Kafka Producer One Application Started ... ")

    # Creating KafkaProducer object
    kafka_producer_obj = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
                                       value_serializer=lambda x: x.encode('utf-8'))
    
    # Path to the CSV file
                # D:\internship-f1soft\day-3 assignment\assignment_2
    # filepath = "D:/internship-f1soft/day-3 assignment/assignment_2/fc_transaction_base.csv"
    filepath = "D:/internship-f1soft/day-3 assignment/assignment_2/transaction_summary.csv"

    
    # Reading the CSV file into a DataFrame using spark

    transaction_df = spark.read.option("header", "true").csv(filepath)
    # transaction_df.show()
    transaction_df_head = transaction_df.limit(10)
    # transaction_df_head.show()

#     enc_df = df.withColumn('encrypted_account_number', expr("base64(aes_encrypt(account_number, '1234567890abcdef', 'ECB', 'PKCS'))")).select("encrypted_account_number")
# enc_df.show(5)

    # encryption 
    enc_df = transaction_df_head.withColumn('encrypted_Account_No', expr("base64(aes_encrypt(`Account No`, '1234567890abcdef', 'ECB', 'PKCS'))")).select("encrypted_Account_No","BALANCE AMT")
    # enc_df.show()


    # Convert pyspark dataframe to a list of dictionaries
    rows = enc_df.collect()

    # Convert each Row object into a dictionary and collect them into a list
    transaction_list = [row.asDict() for row in rows]
    print(transaction_list)


    for message in transaction_list:
        # print(message)
        
        message_fields_value_list = []
       
        message_fields_value_list.append(message["encrypted_Account_No"])
        # message_fields_value_list.append(message["DATE"])     
        # message_fields_value_list.append(message["TRANSACTION DETAILS"])
        # message_fields_value_list.append(message["CHQ.NO."])
        # message_fields_value_list.append(message["VALUE DATE"])
        # message_fields_value_list.append(message[" WITHDRAWAL AMT "])
        # message_fields_value_list.append(message[" DEPOSIT AMT "])
        message_fields_value_list.append(message["BALANCE AMT"])

        # print(message_fields_value_list)

        message = ','.join(str(v) for v in message_fields_value_list) # into a single string, separated by commas.
        # print(message)
        print("Message Type: ", type(message))
        print("Message: ", message)
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)
        # print("next round hai ta")
        time.sleep(1)
        # print("next round")


    print("Kafka Producer Application Completed. ")


Kafka Producer One Application Started ... 
[{'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 1,000,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 2,000,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 2,500,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 5,500,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 6,000,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 6,500,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 7,000,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 7,500,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 8,000,000.00 '}, {'encrypted_Account_No': '7ZT0YgWeCzHzGLOSngychA==', 'BALANCE AMT': ' 8,500,000.00 '}]
Message Type:  <class 'str'>
Message:  7ZT0YgWeCzHzGLOSngychA==, 1,000,000.00 
Messag

In [20]:
df = pd.read_csv("D:/internship-f1soft/day-3 assignment/assignment_2/transaction_summary.csv")

In [21]:
df.head()

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',29-Jun-17,TRF FROM Indiaforensic SERVICES,NaN,29-Jun-17,NaN,"1,000,000.00","1,000,000.00",.
1,409000611074',5-Jul-17,TRF FROM Indiaforensic SERVICES,NaN,5-Jul-17,NaN,"1,000,000.00","2,000,000.00",.
2,409000611074',18-Jul-17,FDRL/INTERNAL FUND TRANSFE,NaN,18-Jul-17,NaN,"500,000.00","2,500,000.00",.
3,409000611074',1-Aug-17,TRF FRM Indiaforensic SERVICES,NaN,1-Aug-17,NaN,"3,000,000.00","5,500,000.00",.
4,409000611074',16-Aug-17,FDRL/INTERNAL FUND TRANSFE,NaN,16-Aug-17,NaN,"500,000.00","6,000,000.00",.
